In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import timedelta, datetime
import psycopg2 as psy
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import requests
from lxml.html.soupparser import fromstring
from queue import LifoQueue
from itertools import zip_longest
import yfinance as yf
from sklearn.utils import shuffle

/usr/local/lib/python3.6/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
!pip install yfinance

     |████████████████████████████████| 5.5MB 5.0MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=cb74d68c182d3d7ad2c2cd6ad744ef8b57c33403a4412f61112f7a0f25498169
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
conn = psy.connect("host=134.122.24.175" + " dbname=sec_edgar" + " user=chubak" +
                         " password=R3n0_Nevada")

cur = conn.cursor()

In [3]:
cur.execute("SELECT * FROM holdings_xml")

all_rec = cur.fetchall()

df_all = pd.DataFrame.from_records(all_rec, columns=[x[0] for x in cur.description])

print(df_all.head())

cur.execute("SELECT * FROM fails_to_deliver")

ftd_rec = cur.fetchall()

df_ftd = pd.DataFrame.from_records(ftd_rec, columns=[x[0] for x in cur.description])

df_ftd

   code                  nameofissuer  ...        date         cik
0   123                  ALPHABET INC  ...  2018-08-14  0001647251
1   124                  ALPHABET INC  ...  2018-08-14  0001647251
2   125           CANADIAN PAC RY LTD  ...  2018-08-14  0001647251
3   126  CHARTER COMMUNICATIONS INC N  ...  2018-08-14  0001647251
4   127                    ALTABA INC  ...  2018-08-14  0001647251

[5 rows x 14 columns]


,id,settlement_date,cusip,ticker,price,quantity,description,name
0,1,2019-03-15,B38564108,EURN,7.98,1067.0,EURONAV NV ANTWERPEN (BELGIUM),cnsfails201903b.txt
1,2,2019-03-15,D18190898,DB,8.91,390205.0,DEUTSCHE BANK AG NAMEN AKT (DE,cnsfails201903b.txt
2,3,2019-03-15,G0129K104,AYR,20.37,17.0,AIRCASTLE LIMITED COM,cnsfails201903b.txt
3,4,2019-03-15,G037AX101,AMBA,43.47,326.0,"AMBARELLA, INC ORD SHS",cnsfails201903b.txt
4,5,2019-03-15,G04415108,ANDA,9.70,25.0,ANDINA ACQUISITION CORP III US,cnsfails201903b.txt
...,...,...,...,...,...,...,...,...
47368,47369,2019-03-29,98979Y106,SRTTY,3.36,1413.0,ZOZO INC ADR(JAPAN),cnsfails201903b.txt
47369,47370,2019-03-29,989817101,ZUMZ,24.17,169.0,ZUMIEZ INC COM,cnsfails201903b.txt
47370,47371,2019-03-29,98983V106,ZUO,19.51,44.0,ZUORA INC CL A COM,cnsfails201903b.txt
47371,47372,2019-03-29,98986M103,ZYXI,4.45,37.0,"ZYNEX, INC. COMMON STOCK (NV)",cnsfails201903b.txt


In [4]:
def previous_quarter(ref):
  if ref.month < 4:
      return datetime(ref.year - 1, 12, 31).date()
  elif ref.month < 7:
      return datetime(ref.year, 3, 31).date()
  elif ref.month < 10:
      return datetime(ref.year, 6, 30).date()
  return datetime(ref.year, 9, 30).date()


df_all["date_trunc"] = pd.to_datetime(df_all["date"]).map(previous_quarter)


print(help);

Type help() for interactive help, or help(object) for help about object.


In [ ]:
df_ftd["settlement_date"] = pd.to_datetime(df_ftd["settlement_date"])
df_ftd['price'] = pd.to_numeric(df_ftd['price'], errors='coerce')

df_sub = df_ftd.loc[:, ["settlement_date", "price"]]

df_sub.groupby("settlement_date").min().rename(columns={"price": "min_price"})

In [5]:
df_cusip_ticker = df_ftd.loc[:, ["cusip", "ticker"]].drop_duplicates()

df_cusip_ticker;

In [6]:
print(df_all.shape)
df_join = pd.merge(df_all, df_cusip_ticker, on=["cusip"], how="inner")
print(df_join.shape)

(139135, 15)
(109855, 16)


In [ ]:
df_joinn = df_join[df_join["nameofissuer"] != 0]

join_tickers = " ".join([x for x in list(df_joinn["ticker"].unique()) if type(x) == str])


join_tickers

In [ ]:
df_joinn

,code,nameofissuer,titleofclass,cusip,value,sshprnamt,sshprnamttype,investmentdiscretion,othermanager,votingauthoritysole,votingauthorityshared,votingauthoritynone,date,cik,date_trunc,ticker
0,123.0,ALPHABET INC,CAP STK CL C,02079K107,3093307.0,2772650.0,SH,SOLE,0,2772650.0,0.0,0.0,2018-08-14,0001647251,2018-06-30,GOOG
1,439.0,ALPHABET INC,CAP STK CL C,02079K107,10349.0,8900.0,SH,SOLE,0,8900.0,0.0,0.0,2020-05-12,0000947996,2020-03-31,GOOG
2,551.0,Alphabet Inc,CAP STK CL C,02079K107,3107195.0,3239665.0,SH,SOLE,0,3079369.0,0.0,160296.0,2017-11-13,0000200217,2017-09-30,GOOG
3,1072.0,ALPHABET INC-CL C,COM,02079K107,97842.0,126768.0,SH,SOLE,0,126213.0,0.0,555.0,2017-02-07,0000732905,2016-12-31,GOOG
4,1126.0,ALPHABET INC,CAP STK CL C,02079K107,120078.0,98505.0,SH,SOLE,0,98505.0,0.0,0.0,2019-11-13,0000883965,2019-09-30,GOOG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139171,139006.0,PROSHARES TR II,ULSHT BLOOMB OIL,74347W668,373.0,25000.0,SH,SOLE,0,25000.0,0.0,0.0,2018-08-13,0001169581,2018-06-30,SCO
139172,138976.0,BIOTIME INC,COM,09066L105,93.0,45000.0,SH,SOLE,0,45000.0,0.0,0.0,2018-08-13,0001169581,2018-06-30,BTX
139173,138982.0,DIREXION SHS ETF TR,DAILY JR BULL 3X,25460E851,1365.0,100000.0,SH,SOLE,0,100000.0,0.0,0.0,2018-08-13,0001169581,2018-06-30,JNUG
139174,139062.0,PowerShares Emerging Mark,SOVEREIGN DEBT,73936T573,556.0,18693.0,SH,SOLE,0,18693.0,0.0,0.0,2013-06-05,0001340807,2013-03-31,0


In [ ]:
ls_mins = []

for name, group in df_join.groupby("date_trunc"):
  print(name)
  price_data =  yf.download(tickers=list(group["ticker"].unique()), start=str(name - timedelta(days=89)), end=str(name))

  for ticker in group["ticker"].unique():
    print(f"{ticker} - {name}")
    min_close = price_data["Close", ticker].min()
    min_low = price_data["Low", ticker].min()

    ls_mins.append((str(name - timedelta(days=90)), name, ticker, min_close, min_low))




df_min = pd.DataFrame.from_records(ls_mins, columns=["start", "end", "ticker", "min_close", "min_low"])

df_min.to_csv("mins.csv")

df_min

In [ ]:
price_dates = [];


for date in df_join["date_trunc"].unique():
  date_str = str(date).split("T")[0]
  date = pd.to_datetime(date)
  date_str_retry = str(date - timedelta(days=1)).split(" ")[0]
  date_str_minone = str(date - timedelta(days=3)).split(" ")[0]
  date_str_addone = str(date + timedelta(days=3)).split(" ")[0]
  symbs = df_join[df_join["date_trunc"] == date]["ticker"].unique()
  data = yf.download(list(symbs), start=date_str_minone, end=date_str_addone)
  for symb in symbs:
    print(f"{symb}-{date_str}")

    try:
      try:
        try:
          try:
            close = data["Close", symb][date_str]
            low = data["Low", symb][date_str]
          except:
            close = data["Close", symb][date_str_retry]
            low = data["Low", symb][date_str_retry]
        except:
          close = data["Close", symb][date_str_addone]
          low = data["Low", symb][date_str_addone]
      except:
        close = data["Close", symb][date_str_minone]
        low = data["Low", symb][date_str_minone]
    except:
      close = 0
      low = 0

    price_dates.append((date, symb, close, low))





In [ ]:
df_2019_1 = df_joinn[df_joinn["date_trunc"].astype(str).str.contains("2018-12-")]

df_2019_1

In [ ]:
ls_cost_test = []

for name, group in df_2019_1.groupby("ticker"):
  print(name)
  try:
    closes = test_data["Adj Close", name]
  except:
    continue
  try:
    min_clost = closes.min()
    print(f"Min close for {name} is {min_clost}")
  except:
    continue

  df_d_sh = pd.DataFrame.from_records(ls_d_sh, columns=["sh", "date"])

  i = 0
  last_sh = df_d_sh.reset_index().loc[0, "sh"]
  last_inc = ""
  for j, sh in enumerate(df_d_sh.loc[:, "sh"]):
    if sh - last_sh > 0:
      inc.append(f"inc-{i}")
      last_inc = "inc"
    elif sh - last_sh < 0:
      inc.append(f"dec-{i}")
      last_inc = "dec"
    elif sh - last_sh > 0 and last_inc == "dec":
      i = j
      inc.append(f"inc-{i}")
      last_inc = "inc"      
    elif sh - last_sh < 0 and last_inc == "inc":
      i = j
      inc.append(f"dec-{i}")
      last_inc = "dec"      
    elif sh - last_sh == 0:
      inc.append(f"sing-{i}")
      i = j

    last_sh = sh

  df_d_sh["inc"] = inc

  for name_inner, group_inner in df_d_sh.groupby("inc"):
    print(name_inner)

    costs = []
    prior_sh = []
    

    if 

    if "inc" in name_inner:
      ls_inner = []

      last_sh = group_inner.reset_index().loc[0, "sh"]
      
      for _, row in group_inner.iloc[1:, :].iterrows():
        try:
          print(f"{last_sh} - {row['sh']} = {last_sh - row['sh']}")
          print((row["date"], name, (last_sh - row["sh"]) * min_clost))
          ls_inner.append((row["date"], name, (last_sh - row["sh"]) * min_clost))
        except:
          continue
        

        last_sh = row["sh"]
        
      ls_cost_test.extend(ls_inner)

    elif "dec" in name_inner:
      ls_inner = []

      last_sh = group_inner.reset_index().loc[0, "sh"]
      
      for _, row in group_inner.iloc[:, :].iterrows():
        try:          
          print((row["date"], name, last_sh * min_clost))
          ls_inner.append((row["date"], name, last_sh * min_clost))
        except:
          continue
        
        last_sh = row["sh"]

      if len(ls_inner) > 0:
        ls_cost_test.extend(ls_inner[-1] * group_inner.shape[0])
      



In [ ]:
test_data["Adj Close", "AABA"]

In [ ]:
df_cost_test = pd.DataFrame.from_records(ls_cost_test, columns=["date", "symbol", "cost"])

pivot = df_cost_test.pivot_table(values=["cost"], index=["date"], columns=["symbol"])

In [ ]:
pivot

In [1]:
ls_cost_test

NameError: ignored

In [ ]:
data = yf.download(tickers=["MSFT", "GOOG"], start="2019-1-5", end="2019-4-5")

In [2]:
data["Low", "GOOG"]

NameError: ignored

In [ ]:
mins_copy = df_min

def minus_one_day(date):
  return date - timedelta(days=1)


mins_copy["start"] = mins_copy["start"].map(minus_one_day)

mins_copy

In [ ]:
mins_copy.to_csv("mins_minus_one.csv")

In [ ]:
df_prc = pd.merge(mins_copy.loc[:, ["end", "ticker", "min_close", "min_low"]].rename(columns={"end": "date_trunc"}), df_join, on=["date_trunc", "ticker"], how="outer")

df_prc

In [7]:
df_prdt = pd.read_csv("/content/price_dates.csv")


df_prdt = df_prdt.drop(columns=["Unnamed: 0"])

df_prdt

,date_trunc,ticker,close_price,low_price
0,2018-06-30,GOOG,1115.650024,1115.000000
1,2018-06-30,GOOGL,1129.189941,1127.020020
2,2018-06-30,CP,183.020004,181.029999
3,2018-06-30,CHTR,293.209991,286.829987
4,2018-06-30,AABA,NaN,NaN
...,...,...,...,...
46395,2013-03-31,MCF,40.090000,39.799999
46396,2013-03-31,LRLCY,31.650000,31.549999
46397,2013-03-31,RAIL,21.820000,21.570000
46398,2013-03-31,KHNGY,21.910000,21.799999


In [11]:
df_mins = pd.read_csv("/content/mins.csv")



df_mins = df_mins.drop(columns=["Unnamed: 0"])
df_mins = df_mins.rename(columns={"end": "date_trunc"})

df_mins

,start,date_trunc,ticker,min_close,min_low
0,2012-12-29,2013-03-31,MSFT,26.459999,26.280001
1,2012-12-29,2013-03-31,MMM,94.669998,93.959999
2,2012-12-29,2013-03-31,AAPL,15.001785,14.964286
3,2012-12-29,2013-03-31,ADP,51.264267,50.702370
4,2012-12-29,2013-03-31,EAT,31.030001,30.850000
...,...,...,...,...,...
46395,2020-03-30,2020-06-30,RADA,3.640000,3.460000
46396,2020-03-30,2020-06-30,SPNE,7.160000,6.835000
46397,2020-03-30,2020-06-30,VICR,37.900002,37.279999
46398,2020-03-30,2020-06-30,NRK,11.710000,11.650000


In [ ]:
fin_list = []
for name, group in df_join.groupby(["cusip", "cik"]):
  group = group.sort_values(by=["date_trunc"], ascending=False)
  last_date = group.reset_index().loc[0, "date_trunc"]
  last_sh = group.reset_index().loc[0, "sshprnamt"]
  last_sell = group.reset_index().loc[0, "date_trunc"]
  last_buy = group.reset_index().loc[0, "date_trunc"]
  try:
    sh_sec = group.reset_index().loc[1, "sshprnamt"]
    if last_sh < sh_sec:
      last_peak = "inc"
    else:
      last_peak = "dec"
  except:
    fin_list.append((name[0], name[1], last_date, last_sh, last_sell, last_buy, 0))
    continue

  

  for index, row in group.reset_index().loc[1:, ["sshprnamt", "date_trunc", "date", "ticker"]].iterrows():   
    print(f"{name[0]}|{name[1]}|{index}")
    current_date = row["date_trunc"]
    price_row = df_prdt[(df_prdt["date_trunc"] == str(current_date)) & (df_prdt["ticker"] == row["ticker"])]
    min_row = df_mins[(df_mins["date_trunc"] == str(current_date)) & (df_mins["ticker"] == row["ticker"])]
    current_sh = row["sshprnamt"]

    if price_row["low_price"] is None:
      continue

    if current_sh > last_sh and last_peak == "inc":
      last_buy = current_date
      fin_list.append((name[0], name[1], row["ticker"], row["date_trunc"], row["sshprnamt"],
                       last_sell, last_buy, pd.Timedelta(last_buy - row["date_trunc"]).days,
                       float(price_row["close_price"]), float(price_row["low_price"]),
                       float(min_row["min_close"]), float(min_row["min_low"])))
      last_peak = "dec"
    elif current_sh < last_sh and last_peak == "dec":
      last_sell = last_date
      fin_list.append((name[0], name[1], row["ticker"], row["date_trunc"],  row["sshprnamt"],
                       last_sell, last_buy, pd.Timedelta(last_buy - row["date_trunc"]).days, 
                       float(price_row["close_price"]), float(price_row["low_price"]),
                       float(min_row["min_close"]), float(min_row["min_low"])))
      last_peak = "inc"
    else:
      fin_list.append((name[0], name[1], row["ticker"], row["date_trunc"], row["sshprnamt"],
                       last_sell, last_buy, pd.Timedelta(last_buy - row["date_trunc"]).days, 
                       float(price_row["close_price"]), float(price_row["low_price"]),
                       float(min_row["min_close"]), float(min_row["min_low"])))

    last_date = current_date
    last_sh = current_sh

   

    


df_hp = pd.DataFrame.from_records(fin_list, columns=["cusip", "cik", "ticker", "date_trunc", "sshprnamt", "last_sell", "last_buy", "holding_period", 
                                                     "close_price", "low_price", "min_low", "min_close"])
df_hp.to_csv("df_hppr.csv")

df_hp

Streaming output truncated to the last 5000 lines.
134429109|0001040273|3
134429109|0001040273|4
134429109|0001040273|5
134429109|0001040273|6
134429109|0001106129|1
134429109|0001106129|2
134429109|0001106129|3
134429109|0001106129|4
134429109|0001106129|5
134429109|0001106129|6
134429109|0001106129|7
134429109|0001106129|8
134429109|0001106129|9
134429109|0001106129|10
134429109|0001106129|11
134429109|0001106129|12
134429109|0001106129|13
134429109|0001106129|14
134429109|0001106129|15
134429109|0001106129|16
134429109|0001106129|17
134429109|0001106129|18
134429109|0001106129|19
134429109|0001106129|20
134429109|0001106129|21
134429109|0001106129|22
134429109|0001314620|1
134429109|0001608108|1
13462K109|0001358706|1
13462K109|0001358706|2
13462K109|0001358706|3
13462K109|0001358706|4
13462K109|0001358706|5
13462K109|0001358706|6
13462K109|0001358706|7
135086106|0001103804|1
135086106|0001135778|1
136375102|0000813917|1
136375102|0000813917|2
136375102|0000813917|3
136375102|000081

In [21]:
df_prdt[(df_prdt["date_trunc"] == "2013-03-31") & (df_prdt["ticker"] == "MSFT")]


,date_trunc,ticker,close_price,low_price
46248,2013-03-31,MSFT,28.610001,28.26


In [14]:
df_prdt = pd.read_csv("/content/price_dates.csv")


df_prdt = df_prdt.drop(columns=["Unnamed: 0"])

df_prdt

,date_trunc,ticker,close_price,low_price
0,2018-06-30,GOOG,1115.650024,1115.000000
1,2018-06-30,GOOGL,1129.189941,1127.020020
2,2018-06-30,CP,183.020004,181.029999
3,2018-06-30,CHTR,293.209991,286.829987
4,2018-06-30,AABA,NaN,NaN
...,...,...,...,...
46395,2013-03-31,MCF,40.090000,39.799999
46396,2013-03-31,LRLCY,31.650000,31.549999
46397,2013-03-31,RAIL,21.820000,21.570000
46398,2013-03-31,KHNGY,21.910000,21.799999


In [ ]:
df_prdt["date_trunc"] = pd.to_datetime(df_prdt["date_trunc"])
df_hp["date_trunc"] = pd.to_datetime(df_hp["date_trunc"])
df_prhp = pd.merge(df_prdt, df_hp, on=["date_trunc"], how="inner")

df_prhp

In [ ]:
df_mins = pd.read_csv("/content/mins.csv")



df_mins = df_mins.drop(columns=["Unnamed: 0"])
df_mins = df_mins.rename(columns={"end": "date_trunc"})



In [ ]:
df_mins["date_trunc"] = pd.to_datetime(df_mins["date_trunc"])
df_join["date_trunc"] = pd.to_datetime(df_join["date_trunc"])

df_min_join = pd.merge(df_join, df_mins.loc[:, ["date_trunc", "ticker", "min_close", "min_low"]], on=["date_trunc", "ticker"])

df_min_join = df_min_join.loc[:, ["cusip", "cik", "date_trunc", "ticker", "value", "sshprnamt", "min_close", "min_low", "titleofclass", "sshprnamttype"]]

print(df_min_join.shape)

df_min_join = df_min_join.drop_duplicates(subset=["cusip", "ticker",  "cik", "date_trunc", "sshprnamt"]).dropna()

df_min_join

In [ ]:
df_min_join["date_trunc"] = pd.to_datetime(df_min_join["date_trunc"])
df_hp["cusip"] = df_hp["cusip"].astype(str)
df_hp["cik"] = df_hp["cik"].astype(str)
df_hp["sshprnamt"] = df_hp["sshprnamt"].astype(str)
df_hp["date_trunc"] = pd.to_datetime(df_hp["date_trunc"])
df_min_join["cusip"] = df_min_join["cusip"].astype(str)
df_min_join["cik"] = df_min_join["cik"].astype(str)
df_min_join["sshprnamt"] = df_min_join["sshprnamt"].astype(str)
df_mjn = pd.merge(df_hp, df_min_join, on=["cusip", "cik", "sshprnamt", "date_trunc"], how="inner")

df_mjn = df_mjn.drop_duplicates().dropna()

df_mjn

In [ ]:
df_mins["date_trunc"] = pd.to_datetime(df_mins["date_trunc"])
df_join["date_trunc"] = pd.to_datetime(df_join["date_trunc"])

df_min_join = pd.merge(df_join, df_mins.loc[:, ["date_trunc", "ticker", "min_close", "min_low"]], on=["date_trunc", "ticker"])

df_min_join = df_min_join.loc[:, ["cusip", "cik", "date_trunc", "ticker", "value", "sshprnamt", "min_close", "min_low", "titleofclass", "sshprnamttype"]]

print(df_min_join.shape)

df_min_join = df_min_join.drop_duplicates(subset=["cusip", "ticker",  "cik", "date_trunc", "sshprnamt"]).dropna()

df_min_join

In [ ]:
df_min_join = pd.read_csv("/content/min_join.csv")
print(df_min_join.shape)


df_min_join = df_min_join.dropna()

print(df_min_join.shape)

df_min_join = df_min_join.drop_duplicates(subset=["cik", "cusip", "date_trunc", "sshprnamt"])

print(df_min_join.shape)

(39319, 19)
(35043, 19)
(32647, 19)


In [ ]:
df_mjn["sshprnamt"] = pd.to_numeric(df_mjn["sshprnamt"])

In [ ]:
df_mjn = df_mjn.sort_values(by=["cusip", "date_trunc"], ascending=False)


row_1 = df_mjn.iloc[0, :]
row_2 = df_mjn.iloc[1, :]
cost_lows = [row_1["sshprnamt"] * row_1["min_low"]]
cost_closes = [row_1["sshprnamt"] * row_1["min_close"]]
last_sh = row_1["sshprnamt"]
sh_array = []
last_peak = "inc" if last_sh < row_2["sshprnamt"] else "dec"
types = []
sh_sum = 0
plateau = True if last_sh == row_2["sshprnamt"] else False

for index, row in df_mjn.iloc[1:, :].iterrows():
  current_sh = row["sshprnamt"]
  date_equals = row["date_trunc"] == row["last_buy"]
  if last_sh > current_sh and not date_equals:
    types.append(f"dec")
    cost_lows.append(cost_lows[index - 1] / ((last_sh) * (current_sh - last_sh) / 1000)  + cost_lows[index - 1])
    cost_closes.append(cost_closes[index - 1] / ((last_sh) * (current_sh - last_sh) / 1000) + cost_closes[index - 1]) 
    last_sh = current_sh
    continue
  elif current_sh > last_sh and not date_equals:
    types.append('inc')
    cost_lows.append(row["min_low"] * ((current_sh - last_sh) / 1000) + cost_lows[index - 1]) 
    cost_closes.append(row["min_close"] * ((current_sh - last_sh)  / 1000) + cost_closes[index - 1])
    last_sh = current_sh
    continue
  elif date_equals:
    cost_lows.append((row["min_low"] * row["sshprnamt"]) / 1000)
    cost_closes.append((row["min_close"] * row["sshprnamt"]) / 1000)
    types.append("de")
    last_sh = current_sh
    continue
  elif last_sh == current_sh:
    cost_lows.append(cost_lows[index - 1])
    cost_closes.append(cost_closes[index - 1])
    types.append("plt")
    last_sh = current_sh
    continue
  else:
    cost_lows.append((row["min_low"] * row["sshprnamt"]) / 1000)
    cost_closes.append((row["min_close"] * row["sshprnamt"]) / 1000)
    print(row)
    types.append("else")
    last_sh = current_sh
    continue

  




In [ ]:
print(len(cost_closes))
print(len(cost_lows))
print(len(types))

In [ ]:
df_cost = df_mjn
types.insert(0, "init")
df_cost["cost_low"] = cost_lows
df_cost["cost_close"] = cost_closes
df_cost["peak_type"] = types

df_cost.loc[3305:3325, ["cik", "cusip", "date_trunc", "last_buy", "min_close", "min_low", "sshprnamt", "cost_close", "value", "cost_low", "peak_type"]]

In [ ]:
sub = df_cost

sub["cost_close"] = sub["cost_close"].astype(object)
sub["cost_low"] = sub["cost_low"].astype(object)

sub

In [ ]:
df_min_join.to_csv('min_join.csv')

In [ ]:
df_cost[df_cost["peak_type"] == "dec-np"]

In [ ]:
df_cost.iloc[51500:51520, :]

In [ ]:
df_cost.to_csv("cost_test.csv")

In [ ]:
df_cost[df_cost["peak_type"] == "de"]

In [ ]:
df_mjn[df_mjn["date_trunc"] == df_mjn["last_buy"]]

In [ ]:
df_cost["cost_close"] = pd.to_numeric(df_cost["cost_close"])

df_cost[["value", "cost_close"]].cov()

In [ ]:
print(df_mjn.shape)
df_mjn = df_mjn[df_mjn["sshprnamt"] != 0]
print(df_mjn.shape)
df_mjn = df_mjn.reset_index()

In [ ]:
plt.hist(df_cost['value'], bins=10)

plt.show()

In [ ]:
df_cost[df_cost["cost_close"] == df_cost["cost_close"].max()]

In [ ]:
df_mjn = df_mjn.drop(columns=["level_0", "index"]).reset_index().drop(columns=["index"])

df_mjn